In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np
import pandas as pd
import seaborn as sns

#for plot with ATLAS styple
import matplotlib.pyplot as plt
from matplotlib import colors

import uproot
import gc

from cells import Cells 

MCdata="cell_analysis_mu60"
MC_root = f"../re21.9/{MCdata}/cells.root"
tree_name = "ntuple"
        
x_bin = np.linspace(-3.15, 3.15, num=33)
y_bin = np.linspace(-5, 5, num=51)

cell_data = Cells(MC_root, unit="GeV")
#cell_imgs, cell_label = cell_data.cimg(x_bin, y_bin, batch_size=500)
cell_imgs, cell_label = cell_data.cimg_et(x_bin, y_bin, batch_size=500)
print(f"Cell image shape: {cell_imgs.shape}")
print(f"Image label shape: {cell_label.shape}")

gc.collect()

input_shapes = cell_imgs.shape[1:]
dim_fletten = np.prod(input_shapes)
print(f"dim_fletten = {dim_fletten}")

Cell image shape: (47000, 50, 32)
Image label shape: (47000, 1)
dim_fletten = 1600


# Weight and bias

# DNN Model

In [2]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import os
import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.layers import Input, InputLayer, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer, GaussianNoise
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import models
from models import MeanSquaredError_ET, MSPE_ET



x_train, x_test, y_train, y_test = train_test_split(cell_imgs, cell_label,
                                                       test_size=0.2, random_state=42,
                                                        shuffle=True
                                                       )
x_train, x_test = x_train.reshape((-1,dim_fletten)), x_test.reshape((-1,dim_fletten))

##############
###########DNN
def hyper_mode(config):
    
    model_DNN = models.DNN(h_dim=config['h_layers'], out_dim=1,
                                     activation=config['activation'], 
                                     L1L2=tf.keras.regularizers.L1L2(l1=0, l2=0.1))
    #compile model
    model_DNN.compile(optimizer=Adam(learning_rate=config['lr']),
              loss=config['loss'], 
              metrics=['mean_absolute_error'] 
             )

    return model_DNN


gc.collect()

#tf.keras.utils.plot_model(model_DNN, expand_nested=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-10-27 14:50:35.231873: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


22

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: ablet. Use `wandb login --relogin` to force relogin


True

In [4]:
# Define sweep config
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_mean_absolute_error'},
    'parameters': 
    {
        'activation': {'values': ['relu', 'linear']},
        'loss' : {'values': ['mean_squared_error', 'mean_absolute_error']},
        'h_layers' : {'values': [[2000], [1500], [1000], [500], [250]] },
        'epochs': {'value': 200},
        'lr': {'max': 0.1, 'min': 0.0001}
     }
}

# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project='hyperTune-NN')

Create sweep with ID: v40zhba2
Sweep URL: https://wandb.ai/ablet/hyperTune-NN/sweeps/v40zhba2


In [5]:
WANDB_PROJECT_NAME = 'hyperTune-NN'
def train():
    with wandb.init(project=WANDB_PROJECT_NAME) as run:
        config = wandb.config
        model = hyper_mode(config)
        
        model.fit(x_train, y_train, 
                        batch_size= 1024,
                        epochs=config['epochs'], 
                        verbose=0, validation_split=0.2,
                        callbacks=[wandb.keras.WandbCallback()]
                       )   
        

wandb.agent(sweep_id=sweep_id, function=train, count=5)

wandb: Agent Starting Run: qd3wfz7e with config:
wandb: 	activation: relu
wandb: 	epochs: 200
wandb: 	h_layers: [2000]
wandb: 	loss: mean_absolute_error
wandb: 	lr: 0.06324541681529575
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


2022-10-27 14:51:05.382591: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_145059-qd3wfz7e/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_145059-qd3wfz7e/files/model-best)... Done. 0.1s


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_145059-qd3wfz7e/files/model-best/assets


...

INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_152017-iawubkx9/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_152017-iawubkx9/files/model-best)... Done. 0.1s


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_152017-iawubkx9/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_152017-iawubkx9/files/model-best)... Done. 0.1s


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▂▃█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mean_absolute_error,▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,118
best_val_loss,34.27179
epoch,199
loss,37.24381
mean_absolute_error,33.04263
val_loss,37.00205


In [6]:
wandb.agent(sweep_id=sweep_id, function=train, count=5)

wandb: Agent Starting Run: chrtabdf with config:
wandb: 	activation: relu
wandb: 	epochs: 200
wandb: 	h_layers: [1000]
wandb: 	loss: mean_squared_error
wandb: 	lr: 0.04685100099611096
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_153316-chrtabdf/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_153316-chrtabdf/files/model-best)... Done. 0.1s


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_153316-chrtabdf/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_153316-chrtabdf/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_160301-plgpzeip/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_160301-plgpzeip/files/model-best)... Done. 0.1s


INFO:tensorflow:Assets written to: /Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_160301-plgpzeip/files/model-best/assets


wandb: Adding directory to artifact (/Users/abletimin/cernbox/Trigger/ml-met-trigger/wandb/run-20221027_160301-plgpzeip/files/model-best)... Done. 0.1s


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▄█▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁
mean_absolute_error,██▇▇▆▆▅▅▅▄▄▄▃▃▃▃▃▃▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁
val_loss,▆▅▅▄▃▃▃▃▃▃▃▃▂▂▃▂▂▄█▂▁▂▂▂▂▂▁▂▁▂▁▁▂▂▂▁▁▂▁▁
val_mean_absolute_error,█▇▇▇▆▆▅▅▅▅▄▄▄▃▄▄▃▃▅▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▂▂▁
best_epoch,195
best_val_loss,36.06547
epoch,199
loss,35.45767
mean_absolute_error,33.77556
val_loss,36.38143
